# Step by step

In [371]:
from pyspark import SparkContext, SparkConf
import numpy as np
import time


sc = SparkContext.getOrCreate()
tr = sc.textFile("datasets/small.txt").zipWithUniqueId()
k = sc.broadcast(5)

In [372]:
cart = tr.cartesian(tr)
cart.saveAsTextFile('cart')

In [373]:
def distances (dt):
    idts = dt[0][1]
    test = np.array(dt[0][0].split(","))
    ts = test[0:11].astype(np.float)
    clts = int(test[11])
    idtr = dt[1][1]
    train = np.array(dt[1][0].split(","))
    tr = train[0:11].astype(np.float)
    cltr = int(train[11])
    
    dist = np.sum((ts-tr)**2)
    
    if idts == idtr:
        dist = float("inf")
    
    return (idts,(dist,cltr,clts))
    

In [374]:
def distances (dt):
    idts = dt[0][1]
    test = dt[0][0].split(",")
    ts = list(map(float, test[0:7]))
    clts = int(test[7])
    idtr = dt[1][1]
    train = dt[1][0].split(",")
    tr = list(map(float, train[0:7]))
    cltr = int(train[7])
    
    dist = sum((p-q)**2 for p, q in zip(ts, tr)) ** .5
    
    if idts == idtr:
        dist = float("inf")
    
    return (idts,(dist,cltr,clts))

In [375]:

start = time.time()
dist = cart.map(distances)
dist.saveAsTextFile('dist')
end = time.time()
print('The time to run is:', end - start)

The time to run is: 0.848015546798706


In [376]:
def create_klist(value):
    x = [(float('inf'),0)]*k.value
    x[0] = value
    return x

def merge_klist(x, value):
    for i in range(len(x)):
        if value[0]<x[i][0]:
            for j in range(len(x)-1,i,-1):
                x[j]=x[j-1]
            x[i]=value
            break
    return x

def merge_combiners(x,y):
    l = 0
    for i in range(len(x)):
        if y[l][0]<x[i][0]:
            for j in range(len(x)-1,i,-1):
                x[j]=x[j-1]
            x[i]=y[l]
            l+=1
    return x

In [377]:
def create_klist(value):
    x = [(float('inf'),0)]*k.value
    x[0] = value
    return x

def merge_klist(x, value):
    for i in range(len(x)):
        if value[0]<x[i][0]:
            x.insert(i,value)
            x.pop()
            break
    return x

def merge_combiners(x,y):
    l = 0
    for i in range(len(x)):
        if y[l][0]<x[i][0]:
            x.insert(i,y[l])
            x.pop()
            l+=1
    return x

In [378]:
k_vals = dist.combineByKey(create_klist, merge_klist, merge_combiners)
k_vals.saveAsTextFile('k_vals')

In [379]:
def guess_class(dt):
    rclass = dt[1][0][2]
    freq = 0
    predict = 0
    for i in range(len(dt[1])):
        tfreq = 1
        tpredict = dt[1][i][1]
        for j in range(i+1,len(dt[1])):
            if tpredict == dt[1][j][1]:
                tfreq +=1
        if tfreq > freq:
            predict = tpredict
            freq = tfreq
            
    
    return (rclass,predict)

In [380]:
guess_class = k_vals.map(guess_class)
guess_class.saveAsTextFile('guess_class')

In [381]:
def correct(dt):
    if dt[0]==dt[1]:
        return 1
    else:
        return 0

In [382]:
correct = guess_class.map(correct)
accuracy = correct.mean()


In [383]:
accum = sc.accumulator(0)
correct.foreach(lambda x: accum.add(x))
accum.value

274

In [384]:
accuracy

0.8154761904761905

# Measuring time

In [9]:
from pyspark import SparkContext, SparkConf
import numpy as np
import time




def distances (dt):
    idts = dt[0][1]
    test = dt[0][0].split(",")
    ts = list(map(float, test[0:11]))
    clts = int(test[11])
    idtr = dt[1][1]
    train = dt[1][0].split(",")
    tr = list(map(float, train[0:11]))
    cltr = int(train[11])
    
    dist = sum((p-q)**2 for p, q in zip(ts, tr)) ** .5
    
    if idts == idtr:
        dist = float("inf")
    
    return (idts,(dist,cltr,clts))
    
def create_klist(value):
    x = [(float('inf'),0)]*k.value
    x[0] = value
    return x

def merge_klist(x, value):
    for i in range(len(x)):
        if value[0]<x[i][0]:
            x.insert(i,value)
            x.pop()
            break
    return x

def merge_combiners(x,y):
    l = 0
    for i in range(len(x)):
        if y[l][0]<x[i][0]:
            x.insert(i,y[l])
            x.pop()
            l+=1
    return x

def guess_class(dt):
    rclass = dt[1][0][2]
    freq = 0
    predict = 0
    for i in range(len(dt[1])):
        tfreq = 1
        tpredict = dt[1][i][1]
        for j in range(i+1,len(dt[1])):
            if tpredict == dt[1][j][1]:
                tfreq +=1
        if tfreq > freq:
            predict = tpredict
            freq = tfreq
            
    
    return (rclass,predict)

start = time.time()
sc = SparkContext.getOrCreate()
tr = sc.textFile("datasets/medium.txt").zipWithUniqueId()
k = sc.broadcast(1)

cart = tr.cartesian(tr)
dist = cart.map(distances)
dist.saveAsTextFile('dist')
end = time.time()
print('The time to run is:', end - start)
'''

k_vals = dist.combineByKey(create_klist, merge_klist, merge_combiners)
guess_class = k_vals.map(guess_class)



def correct(dt):
    if dt[0]==dt[1]:
        return 1
    else:
        return 0
    
correct = guess_class.map(correct)
accuracy = correct.mean()
end = time.time()
print('The time to run is:', end - start)
print(accuracy)
'''

The time to run is: 164.25576519966125


"\n\nk_vals = dist.combineByKey(create_klist, merge_klist, merge_combiners)\nguess_class = k_vals.map(guess_class)\n\n\n\ndef correct(dt):\n    if dt[0]==dt[1]:\n        return 1\n    else:\n        return 0\n    \ncorrect = guess_class.map(correct)\naccuracy = correct.mean()\nend = time.time()\nprint('The time to run is:', end - start)\nprint(accuracy)\n"

In [396]:
from pyspark import SparkContext, SparkConf
import numpy as np
import time

start = time.time()


def distances (dt):
    idts = dt[0][1]
    test = dt[0][0].split(",")
    ts = list(map(float, test[0:7]))
    clts = int(test[7])
    idtr = dt[1][1]
    train = dt[1][0].split(",")
    tr = list(map(float, train[0:7]))
    cltr = int(train[7])
    
    dist = sum((p-q)**2 for p, q in zip(ts, tr)) ** .5
    
    if idts == idtr:
        dist = float("inf")
    
    return (idts,(dist,cltr,clts))
    
def create_klist(value):
    x = [(float('inf'),0)]*k.value
    x[0] = value
    return x

def merge_klist(x, value):
    for i in range(len(x)):
        if value[0]<x[i][0]:
            for j in range(len(x)-1,i,-1):
                x[j]=x[j-1]
            x[i]=value
            break
    return x

def merge_combiners(x,y):
    l = 0
    for i in range(len(x)):
        if y[l][0]<x[i][0]:
            for j in range(len(x)-1,i,-1):
                x[j]=x[j-1]
            x[i]=y[l]
            l+=1
    return x

def guess_class(dt):
    rclass = dt[1][0][2]
    freq = 0
    predict = 0
    for i in range(len(dt[1])):
        tfreq = 1
        tpredict = dt[1][i][1]
        for j in range(i+1,len(dt[1])):
            if tpredict == dt[1][j][1]:
                tfreq +=1
        if tfreq > freq:
            predict = tpredict
            freq = tfreq
            
    
    return (rclass,predict)

sc = SparkContext.getOrCreate()
tr = sc.textFile("datasets/small.txt").zipWithUniqueId()
k = sc.broadcast(1)

cart = tr.cartesian(tr)
dist = cart.map(distances)
k_vals = dist.combineByKey(create_klist, merge_klist, merge_combiners)
guess_class = k_vals.map(guess_class)



def correct(dt):
    if dt[0]==dt[1]:
        return 1
    else:
        return 0
    
correct = guess_class.map(correct)
accuracy = correct.mean()
end = time.time()
print('The time to run is:', end - start)
print(accuracy)

The time to run is: 0.9075469970703125
0.7916666666666665


# Second approach

In [363]:
from pyspark import SparkContext, SparkConf
import numpy as np
import time


sc = SparkContext.getOrCreate()
ts = sc.textFile("datasets/small.txt").zipWithUniqueId()\
    .map(lambda line: (line[0].split(','), line[1]))\
    .map(lambda line: (list(map(float, line[0])),line[1]))

tr = ts.collect()
k = sc.broadcast(5)


In [364]:
def distances (dt):
    idts = dt[1]
    ts = list(map(float, dt[0][0:7]))
    cltr = int(dt[0][7])
    
    
    x = [(float('inf'),0)]*k.value
    
    for i in tr:
        if not idts == i[1]:
            dist = sum((p-q)**2 for p, q in zip(i[0][0:7], ts)) ** .5
            for j in range(len(x)):
                if dist < x[j][0]:
                    for z in range(len(x)-1,j,-1):
                        x[z]=x[z-1]
                    x[j]=(dist,i[0][7])
                    break
                
    return (cltr,x)

In [365]:

k_vals = ts.map(distances)
k_vals.saveAsTextFile('k_vals')


In [366]:
def guess_class(dt):
    rclass = dt[0]
    freq = 0
    predict = 0
    for i in range(len(dt[1])):
        tfreq = 1
        tpredict = dt[1][i][1]
        for j in range(i+1,len(dt[1])):
            if tpredict == dt[1][j][1]:
                tfreq +=1
        if tfreq > freq:
            predict = tpredict
            freq = tfreq
    return (rclass,predict)

In [367]:
guess_class = k_vals.map(guess_class)
guess_class.saveAsTextFile('guess_class')

In [368]:
def correct(dt):
    if dt[0]==dt[1]:
        return 1
    else:
        return 0

correct = guess_class.map(correct)
accuracy = correct.mean()


In [369]:
accum = sc.accumulator(0)
correct.foreach(lambda x: accum.add(x))
accum.value

274

In [370]:
accuracy

0.815476190476191

In [392]:
from pyspark import SparkContext, SparkConf
import numpy as np
import time

start = time.time()
def distances (dt):
    idts = dt[1]
    ts = list(map(float, dt[0][0:7]))
    cltr = int(dt[0][7])
    
    
    x = [(float('inf'),0)]*k.value
    
    for i in tr:
        if not idts == i[1]:
            dist = sum((p-q)**2 for p, q in zip(i[0][0:7], ts)) ** .5
            for j in range(len(x)):
                if dist < x[j][0]:
                    for z in range(len(x)-1,j,-1):
                        x[z]=x[z-1]
                    x[j]=(dist,i[0][7])
                    break
                
    return (cltr,x)


def guess_class(dt):
    rclass = dt[0]
    freq = 0
    predict = 0
    for i in range(len(dt[1])):
        tfreq = 1
        tpredict = dt[1][i][1]
        for j in range(i+1,len(dt[1])):
            if tpredict == dt[1][j][1]:
                tfreq +=1
        if tfreq > freq:
            predict = tpredict
            freq = tfreq
    return (rclass,predict)

def correct(dt):
    if dt[0]==dt[1]:
        return 1
    else:
        return 0


sc = SparkContext.getOrCreate()
ts = sc.textFile("datasets/small.txt").zipWithUniqueId()\
    .map(lambda line: (line[0].split(','), line[1]))\
    .map(lambda line: (list(map(float, line[0])),line[1]))

tr = ts.collect()
k = sc.broadcast(5)


k_vals = ts.map(distances)
guess_class = k_vals.map(guess_class)
correct = guess_class.map(correct)
accuracy = correct.mean()
end = time.time()
print('The time to run is:', end - start)
print('accuracy')

The time to run is: 0.4284708499908447


0.815476190476191

In [397]:
from pyspark import SparkContext, SparkConf
import numpy as np
import time

start = time.time()
def distances (dt):
    idts = dt[1]
    ts = list(map(float, dt[0][0:11]))
    cltr = int(dt[0][11])
    
    
    x = [(float('inf'),0)]*k.value
    
    for i in tr:
        if not idts == i[1]:
            dist = sum((p-q)**2 for p, q in zip(i[0][0:11], ts)) ** .5
            for j in range(len(x)):
                if dist < x[j][0]:
                    for z in range(len(x)-1,j,-1):
                        x[z]=x[z-1]
                    x[j]=(dist,i[0][11])
                    break
                
    return (cltr,x)


def guess_class(dt):
    rclass = dt[0]
    freq = 0
    predict = 0
    for i in range(len(dt[1])):
        tfreq = 1
        tpredict = dt[1][i][1]
        for j in range(i+1,len(dt[1])):
            if tpredict == dt[1][j][1]:
                tfreq +=1
        if tfreq > freq:
            predict = tpredict
            freq = tfreq
    return (rclass,predict)

def correct(dt):
    if dt[0]==dt[1]:
        return 1
    else:
        return 0


sc = SparkContext.getOrCreate()
ts = sc.textFile("datasets/medium.txt").zipWithUniqueId()\
    .map(lambda line: (line[0].split(','), line[1]))\
    .map(lambda line: (list(map(float, line[0])),line[1]))

tr = ts.collect()
k = sc.broadcast(5)


k_vals = ts.map(distances)
guess_class = k_vals.map(guess_class)
correct = guess_class.map(correct)
accuracy = correct.mean()
end = time.time()
print('The time to run is:', end - start)
print('accuracy')

The time to run is: 80.40959596633911
accuracy
